<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/Milena/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install at every runtime?-prob yes
!pip install checklist
!pip install torch

#should i install it every time?-yes
!pip install simpletransformers 


     |████████████████████████████████| 12.1 MB 13.0 MB/s 
     |████████████████████████████████| 4.2 MB 28.5 MB/s 
     |████████████████████████████████| 22.3 MB 2.1 MB/s 
     |████████████████████████████████| 167 kB 35.7 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 6.6 MB 41.9 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |████████████████████████████████| 5.6 MB 41.5 MB/s 
     |████████████████████████████████| 5.6 MB 22.6 MB/s 
     |████████████████████████████████| 419 kB 47.5 MB/s 
     |████████████████████████████████| 104 kB 42.3 MB/s 
     |████████████████████████████████| 3.6 MB 46.6 MB/s 
  Created wheel for checklist: filename=checklist-0.0.11-py3-none-any.whl size=12165635 sha256=21076570c56b9a41a3a0e6afdc023237e42d41366bedbec4acc03a0364723b4b
  Stored in directory: /root/.cache/pip/wheels/6a/8a/07/6446879be434879c27671c83443727d74

In [13]:
import checklist
import torch
import spacy
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#$loading eng
nlp = spacy.load("en_core_web_sm")

In [2]:
# from platform import python_version #to check version
# python_version()

import pandas as pd
import checklist
from checklist.perturb import Perturb
#from checklist.perturb import add_negation
import nltk #; nltk.download('omw-1.4')
#nltk.download()

In [14]:
# stolen from E6
#setting up google drive and loading diagnostic test data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv")
url = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
#data = pd.read_csv(url)

#$loading eng
#nlp = spacy.load("en_core_web_sm")

Mounted at /content/drive


In [4]:
#loading finetuned MODEL FROM DRIVE
bert_mod = ClassificationModel("bert", "/content/drive/MyDrive/NLP2/trained_models2")

In [27]:
np.random.seed(42)

#adding typos from tutorial
from checklist.perturb import Perturb

pdata = list(data['text'])
ret = Perturb.perturb(pdata, Perturb.add_typos)
ret.data 

[['@USER @USER Who the hell does he think he is?',
  '@USER @USER Who the hell does he thinkh e is?'],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  '#BREAKING. #Greece: Molotov cocktails fly after proetst honouring killed antifa arti... URL via @USER URL'],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges aaginst a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can 

In [6]:
#keeping only the typos
typo_lst=[]
for i in range(len(ret.data)):
  typo_lst.append(ret.data[i][1])

(typo_lst)

['@USER @USER Who the hell does he thinkh e is?',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URLv ia @USER URL',
 '#OrrinHatch I can’t believe this sexist , cleuless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes oyu can choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 '0-1 lsot my acca on the first fucking fight cba',
 '#Bakersfield is why we need gun control! Sorry for the vitcims other than the gunman himself I hope he rots in hell!',
 '#Christian #America – If we go by #Trump’s example, where liebrals support open borders, I guess conserva

In [17]:
#geting predictions on the negations
prediction_typo, out_nr_typo = bert_mod.predict(typo_lst)
prediction_reg, out_nr_reg = bert_mod.predict(pdata)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [19]:
# quantitative analysis
def precision(true_pos, false_pos):
    if true_pos + false_pos != 0:
        prec = true_pos/(true_pos + false_pos)
    else:
        prec = 0
    return prec

def recall(true_pos, false_neg):
    if true_pos + false_neg != 0:
        rec = true_pos/(true_pos + false_neg)
    else:
        rec = 0
    return rec

def f1(precision, recall):
    if precision + recall != 0:
        f1_score = 2*((precision * recall)/(precision + recall))
    else:
        f1_score = 0
    return f1_score

In [18]:
summary = pd.DataFrame([typo_lst, prediction_typo, prediction_reg]).T
summary

,0,1,2
0,@USER @USER Who the hell does he thinkh e is?,0,0
1,#BREAKING. #Greece: Molotov cocktails fly afte...,0,0
2,"#OrrinHatch I can’t believe this sexist , cleu...",0,0
3,@USER @USER I'll use that one the next time im...,0,0
4,0-1 lsot my acca on the first fucking fight cba,0,0
...,...,...,...
95,@USER He iso bviously getting suspended. He is...,0,0
96,#Canada - EXCLUSIVE: #Trudeau #Liberalsl eave ...,0,0
97,@USER @USER ...tahn why did you show us how ho...,0,0
98,@USER@ USER @USER You have yet to answer what ...,0,0


In [26]:
pred0_acc0 = 0
pred1_acc0 = 0
pred0_acc1 = 0
pred1_acc1 = 0

for i in range(len(typo_lst)):
  if prediction_reg[i] == 0 and prediction_typo[i] == 0:
    pred0_acc0 += 1
  elif prediction_reg[i] == 1 and prediction_typo[i] == 0:
    pred0_acc1 += 1
  elif prediction_reg[i] == 0 and prediction_typo[i] == 1:
    pred1_acc0 += 1
  elif prediction_reg[i] == 1 and prediction_typo[i] == 1:
    pred1_acc1 += 1

#print(pred0_acc0, pred1_acc1, pred0_acc1, pred1_acc1)

precision_0 = precision(pred0_acc0, pred0_acc1)
recall_0 = recall(pred0_acc0, pred1_acc0)
f1_0 = f1(precision_0, recall_0)
print(precision_0, recall_0, f1_0)

precision_1 = precision(pred1_acc1, pred1_acc0)
recall_1 = recall(pred1_acc1, pred0_acc1)
f1_1 = f1(precision_1, recall_1)
print(precision_1, recall_1, f1_1)

macro_precision = (precision_0+precision_1)/2
macro_recall = (recall_0 + recall_1)/2
macro_f1 = (f1_0 + f1_1)/2
print(macro_precision, macro_recall, macro_f1)

1.0 1.0 1.0
0 0 0
0.5 0.5 0.5
